<a href="https://colab.research.google.com/github/wander-asb/US_Accidents/blob/main/Us.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Leitura de arquivos

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('Inicializando com Spark').getOrCreate()

In [ ]:
spark

In [6]:
from google.colab                import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
! mkdir ~/.kaggle

In [8]:
! cp /content/gdrive/MyDrive/kaggle_api/kaggle.json ~/.kaggle/kaggle.json

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
! kaggle datasets download -d sobhanmoosavi/us-accidents

100% 652M/653M [00:33<00:00, 25.5MB/s]
100% 653M/653M [00:33<00:00, 20.4MB/s]


In [11]:
! unzip /content/us-accidents.zip

Archive:  /content/us-accidents.zip
  inflating: US_Accidents_March23.csv  


In [12]:
data_dir = '/content/US_Accidents_March23.csv'

## Leitura de dados

In [28]:
us_accident =  spark.read.csv(data_dir, header=True, inferSchema=True)

In [33]:
us_accident.select('Severity')

DataFrame[Severity: int]